# 1.Extracting the Data through webscraping the Wikipedia link:

In [19]:
import pandas as pd # library for data analsysis
from pandas import DataFrame
import json # library to handle JSON files
import requests # library to handle requests
from bs4 import BeautifulSoup
import xml

In [20]:
# Getting Beautiful soup to extract the data in the form of a table
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

In [21]:
# Getting the content of each row of the table into a list
row_data=[]
for row in table.find_all('td'):
    row_data.append(row.text.strip())

In [22]:
# Cretaing a list to store our data:

List1=[]
List2=['M7R','M5W','M7Y']
for i in range(len(row_data)):
    if row_data[i][3:]!='Not assigned':
        a=row_data[i].split('(')[0][:3]
        b=row_data[i].split('(')[0][3:]
        c=row_data[i].split('(')[1].replace(')','')
        if row_data[i].split('(')[0][:3] in List2:
            if row_data[i].split('(')[0][:3]==List2[0]:
                    a=row_data[i].split('(')[0][:3]
                    b='Mississauga'
                    c='Mississauga'
            if row_data[i].split('(')[0][:3]==List2[1]:
                    a=row_data[i].split('(')[0][:3]
                    b='Downtown Toronto'
                    c='Downtown Toronto'
            if row_data[i].split('(')[0][:3]==List2[2]:
                    a=row_data[i].split('(')[0][:3]
                    b='East Toronto'
                    c='East Toronto'
        dict={'Postalcode':a,'Borough':b,'Neighborhood':c}
        List1.append(dict)    

In [23]:
#Creating our dataframe based on the extracted data:

column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(List1,columns = column_names)
df

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,East Toronto
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [24]:
df.shape

(103, 3)

# 2. Importing the csv file conatining the latitudes and longitudes for various neighbourhoods in Canada


In [25]:
lat = pd.read_csv('https://cocl.us/Geospatial_data')
lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
#Renaming the column prior to merging
df.rename(columns={'Postalcode':'Postal Code'},inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


In [27]:
# Merging both tables
df = pd.merge(df, lat ,on='Postal Code')

In [28]:
#The final table with the requiered data:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Clustering of the boroughs that contain the word Toronto 


In [29]:
df2 = df[df['Borough'].str.contains('Toronto',regex=False)]
df2.shape

(39, 5)

In [30]:
# Importing necessary libraries:

!pip install numpy
!conda install -c conda-forge folium=0.5.0 --yes
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans 
%matplotlib inline

Solving environment: done

# All requested packages already installed.



In [31]:

map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df2['Latitude'],df2['Longitude'],df2['Borough'],df2['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [32]:
k=5
toronto_clustering = df2.drop(['Postal Code','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df2.insert(0, 'Cluster Labels', kmeans.labels_)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  from ipykernel import kernelapp as app


In [33]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood'], df2['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters